# Quantum Error Mitigation

In [1]:
!pip install pennylane torch numpy pandas seaborn matplotlib scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 855.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [2]:
!pip install ignite torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 10.7 MB/s eta 0:00:00


In [3]:
# Pennylane modules
import pennylane as qml
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch import nn
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, auc

In [28]:
import time

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Import

In [33]:
IEEE_train = pd.read_csv("/content/drive/MyDrive/Zenqor/datasets/v2/IEEE-CIS/train_data.csv")
IEEE_test = pd.read_csv("/content/drive/MyDrive/Zenqor/datasets/v2/IEEE-CIS/test_data.csv")

## Dataset Truncate

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()

In [34]:
IEEE_train = IEEE_train[:10000]
IEEE_test = IEEE_test[:10000]

In [35]:
IEEE_train = scaler.fit_transform(IEEE_train)
IEEE_test = scaler.fit_transform(IEEE_test)

## Testing data(s):

In [36]:
IEEE_train.shape, IEEE_test.shape

((10000, 51), (10000, 51))

In [37]:
corr_matrix = IEEE_train.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, cmap='coolwarm', fmt=".2f", square=True)
plt.title("Correlation Matrix")
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'corr'

# Device

In [16]:
device = "cuda" if torch.cuda.is_available else "cpu"

device

'cuda'

In [17]:
n_qubits = 8
q_depth = 3

In [18]:
dev = qml.device('lightning.qubit', wires=n_qubits)

dev

<lightning.qubit device (wires=8) at 0x7ed16cc29d50>

# Quantum Model (No QEM)

In [19]:
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates."""
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis."""
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT."""
    for i in range(0, nqubits - 1, 2):  # Even indices
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Odd indices
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat):
    """
    Variational quantum circuit.
    """
    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from |+> state
    H_layer(n_qubits)

    # Embed input features
    RY_layer(q_input_features)

    # Apply trainable layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Return expectation values
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)]
    # return qml.expval(qml.PauliZ(0))

class DressedQuantumNet(nn.Module):
    """
    Torch module for the dressed quantum network.
    """
    def __init__(self, input_shape=51, dataset_idx=1):
        super().__init__()
        self.n_qubits = n_qubits
        self.q_depth = q_depth
        self.q_delta = 0.01
        # self.pre_net = nn.Linear(input_shape, self.n_qubits)
        self.q_params = nn.Parameter(self.q_delta * torch.randn(self.q_depth * self.n_qubits))
        # self.post_net = nn.Linear(self.n_qubits, 7)

        if (dataset_idx == 1) or (dataset_idx == 2):
            self.pre_net = nn.Sequential(
                            nn.Linear(input_shape, 64),
                            nn.ReLU(),
                            nn.Linear(64, 64),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(64, 32),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(32, 16),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(16, 8)
                        )
        elif dataset_idx == 3:
            self.pre_net = nn.Sequential(
                            nn.Linear(input_shape, 16),
                            nn.ReLU(),
                            nn.Linear(16, 16),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(16, 8)
                        )

        self.post_net = nn.Sequential(
                            nn.Linear(n_qubits, 8),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(8, 4),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(4, 1)
                        )

    def forward(self, input_features):
        """
        Forward pass through the dressed quantum network.
        """
        # Preprocessing input to reduce dimensions
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply quantum circuit
        q_out = []
        for elem in q_in:
            elem = elem.clone().detach().to(self.q_params.device)  # Ensure it's on the correct device and detached
            q_out_elem = quantum_net(elem, self.q_params)  # Output as list

            # Convert list to tensor if it's not already
            q_out_elem_tensor = torch.tensor(q_out_elem, dtype=torch.float32, device=self.q_params.device)

            # Apply linear transformation to match 2 output classes (if needed)
            # q_out_elem = torch.nn.Linear(q_out_elem_tensor.size(-1), 2)(q_out_elem_tensor)  # Adjust size to 2
            q_out_elem = self.post_net(q_out_elem_tensor)

            q_out.append(q_out_elem)

        # Stack and process
        q_out = torch.stack(q_out, dim=0)  # Ensure this is stacked correctly
        # print("q_out after loop:", q_out.shape)

        # q_out = q_out.to(self.q_params.device, dtype=torch.float32, requires_grad=True)

        return torch.sigmoid(q_out)

In [20]:
model_IEEE = DressedQuantumNet(input_shape=50, dataset_idx=1)

# Dataset Preparation

In [38]:
X_IEEE_train = IEEE_train[:, :-1]
y_IEEE_train = IEEE_train[:, -1]

X_IEEE_test = IEEE_test[:, :-1]
y_IEEE_test = IEEE_test[:, -1]

In [39]:
class Dataset(Dataset):
    def __init__(self, feature_cols, label_col):
        self.features = feature_cols.astype(np.float32)
        self.labels = label_col.astype(np.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx])
        y = torch.tensor(self.labels[idx])
        return x, y

In [40]:
IEEE_dataset_train = Dataset(X_IEEE_train, y_IEEE_train)

In [41]:
IEEE_dataset_test = Dataset(X_IEEE_test, y_IEEE_test)

In [42]:
train_loader_IEEE = DataLoader(IEEE_dataset_train, batch_size=32, shuffle=True)
test_loader_IEEE = DataLoader(IEEE_dataset_test, batch_size=32, shuffle=False)

# Training Model (No QEM)

In [43]:
optimizer = optim.Adam(model_IEEE.parameters(), lr=0.001)
criterion = nn.BCELoss()
loss_fn = torch.nn.BCELoss()

In [44]:
epoch_losses = []
epoch_accuracies = []

for epoch in range(20):
    model_IEEE.train()
    total_correct = 0
    total_samples = 0
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader_IEEE:
        X_batch = X_batch.to(model_IEEE.q_params.device)
        y_batch = y_batch.to(model_IEEE.q_params.device).unsqueeze(1)

        optimizer.zero_grad()
        output = model_IEEE(X_batch)
        loss = loss_fn(output, y_batch)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        epoch_loss += loss.item() * X_batch.size(0)

        # Convert predictions to binary (assuming binary classification)
        preds = (output > 0.5).float()
        total_correct += (preds == y_batch).sum().item()
        total_samples += y_batch.size(0)

    avg_loss = epoch_loss / total_samples
    accuracy = total_correct / total_samples

    epoch_losses.append(avg_loss)
    epoch_accuracies.append(accuracy)

    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# Plotting
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epoch_losses, label="Loss")
plt.title("Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epoch_accuracies, label="Accuracy", color='green')
plt.title("Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1, Loss: 0.6016, Accuracy: 0.4992
Epoch 2, Loss: 0.3833, Accuracy: 0.9465
Epoch 3, Loss: 0.3385, Accuracy: 0.9617


KeyboardInterrupt: 

# Model with ZNE

In [49]:
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates."""
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis."""
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT."""
    for i in range(0, nqubits - 1, 2):  # Even indices
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Odd indices
        qml.CNOT(wires=[i, i + 1])

# Define separate QNodes for different noise levels
@qml.qnode(dev, interface="torch")
def quantum_net_factor_1(q_input_features, q_weights_flat):
    """
    Variational quantum circuit with original noise level.
    """
    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from |+> state
    H_layer(n_qubits)

    # Embed input features
    RY_layer(q_input_features)

    # Apply trainable layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Return expectation values
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)]

@qml.qnode(dev, interface="torch")
def quantum_net_factor_3(q_input_features, q_weights_flat):
    """
    Variational quantum circuit with 3x noise level.
    """
    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from |+> state
    H_layer(n_qubits)

    # Embed input features
    RY_layer(q_input_features)

    # Apply trainable layers with 3x noise
    for k in range(q_depth):
        # Original entangling layer
        entangling_layer(n_qubits)

        # Add 2 more identity operations (CNOT pairs) to increase noise
        for _ in range(2):
            for i in range(0, n_qubits - 1, 2):
                qml.CNOT(wires=[i, i + 1])
                qml.CNOT(wires=[i, i + 1])
            for i in range(1, n_qubits - 1, 2):
                qml.CNOT(wires=[i, i + 1])
                qml.CNOT(wires=[i, i + 1])

        RY_layer(q_weights[k])

    # Return expectation values
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)]

@qml.qnode(dev, interface="torch")
def quantum_net_factor_5(q_input_features, q_weights_flat):
    """
    Variational quantum circuit with 5x noise level.
    """
    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from |+> state
    H_layer(n_qubits)

    # Embed input features
    RY_layer(q_input_features)

    # Apply trainable layers with 5x noise
    for k in range(q_depth):
        # Original entangling layer
        entangling_layer(n_qubits)

        # Add 4 more identity operations (CNOT pairs) to increase noise
        for _ in range(4):
            for i in range(0, n_qubits - 1, 2):
                qml.CNOT(wires=[i, i + 1])
                qml.CNOT(wires=[i, i + 1])
            for i in range(1, n_qubits - 1, 2):
                qml.CNOT(wires=[i, i + 1])
                qml.CNOT(wires=[i, i + 1])

        RY_layer(q_weights[k])

    # Return expectation values
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)]

def zne_extrapolate(results, scales=[1, 3, 5]):
    """
    Perform Zero-Noise Extrapolation using linear extrapolation.
    Converts all inputs to standard Python/NumPy types before processing.

    Args:
        results: List of results at different noise scales
        scales: The noise scale factors used

    Returns:
        Extrapolated result at zero noise
    """
    import numpy as np

    # Ensure all values are NumPy arrays or Python floats
    x = np.array(scales)
    y = np.array([float(val) for val in results])

    # Fit linear regression
    coeffs = np.polyfit(x, y, len(x)-1)

    # Extrapolate to zero noise
    return float(np.polyval(coeffs, 0))

def quantum_net(q_input_features, q_weights_flat):
    """
    Wrapper function for ZNE-enhanced quantum network.
    """
    # If not in training mode, just use the original circuit
    if not torch.is_grad_enabled():
        return quantum_net_factor_1(q_input_features, q_weights_flat)

    # Run circuits with different noise factors
    results_1 = quantum_net_factor_1(q_input_features, q_weights_flat)
    results_3 = quantum_net_factor_3(q_input_features, q_weights_flat)
    results_5 = quantum_net_factor_5(q_input_features, q_weights_flat)

    # Extract values to Python lists/floats before ZNE
    results_1 = [r.item() if hasattr(r, 'item') else float(r) for r in results_1]
    results_3 = [r.item() if hasattr(r, 'item') else float(r) for r in results_3]
    results_5 = [r.item() if hasattr(r, 'item') else float(r) for r in results_5]

    # Apply ZNE to each qubit measurement separately
    zne_results = []
    for i in range(len(results_1)):
        qubit_results = [results_1[i], results_3[i], results_5[i]]
        extrapolated = zne_extrapolate(qubit_results)
        zne_results.append(extrapolated)

    return zne_results

class ZNEQuantumNet(nn.Module):
    """
    Torch module for the dressed quantum network.
    """
    def __init__(self, input_shape=51, dataset_idx=1):
        super().__init__()
        self.n_qubits = n_qubits
        self.q_depth = q_depth
        self.q_delta = 0.01
        # self.pre_net = nn.Linear(input_shape, self.n_qubits)
        self.q_params = nn.Parameter(self.q_delta * torch.randn(self.q_depth * self.n_qubits))
        # self.post_net = nn.Linear(self.n_qubits, 7)

        if (dataset_idx == 1) or (dataset_idx == 2):
            self.pre_net = nn.Sequential(
                            nn.Linear(input_shape, 64),
                            nn.ReLU(),
                            nn.Linear(64, 64),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(64, 32),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(32, 16),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(16, 8)
                        )
        elif dataset_idx == 3:
            self.pre_net = nn.Sequential(
                            nn.Linear(input_shape, 16),
                            nn.ReLU(),
                            nn.Linear(16, 16),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(16, 8)
                        )

        self.post_net = nn.Sequential(
                            nn.Linear(n_qubits, 8),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(8, 4),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(4, 1)
                        )

    def forward(self, input_features):
        """
        Forward pass through the dressed quantum network.
        """
        # Preprocessing input to reduce dimensions
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply quantum circuit
        q_out = []
        for elem in q_in:
            elem = elem.clone().detach().to(self.q_params.device)  # Ensure it's on the correct device and detached
            q_out_elem = quantum_net(elem, self.q_params)  # Output as list

            # Convert list to tensor if it's not already
            q_out_elem_tensor = torch.tensor(q_out_elem, dtype=torch.float32, device=self.q_params.device)

            # Apply linear transformation to match 2 output classes (if needed)
            # q_out_elem = torch.nn.Linear(q_out_elem_tensor.size(-1), 2)(q_out_elem_tensor)  # Adjust size to 2
            q_out_elem = self.post_net(q_out_elem_tensor)

            q_out.append(q_out_elem)

        # Stack and process
        q_out = torch.stack(q_out, dim=0)  # Ensure this is stacked correctly
        # print("q_out after loop:", q_out.shape)

        # q_out = q_out.to(self.q_params.device, dtype=torch.float32, requires_grad=True)

        return torch.sigmoid(q_out)

In [50]:
model_IEEE_ZNE = ZNEQuantumNet(input_shape=50, dataset_idx=1)

In [51]:
epoch_losses = []
epoch_accuracies = []

for epoch in range(20):
    model_IEEE_ZNE.train()
    total_correct = 0
    total_samples = 0
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader_IEEE:
        X_batch = X_batch.to(model_IEEE_ZNE.q_params.device)
        y_batch = y_batch.to(model_IEEE_ZNE.q_params.device).unsqueeze(1)

        optimizer.zero_grad()
        output = model_IEEE_ZNE(X_batch)
        loss = loss_fn(output, y_batch)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        epoch_loss += loss.item() * X_batch.size(0)

        # Convert predictions to binary (assuming binary classification)
        preds = (output > 0.5).float()
        total_correct += (preds == y_batch).sum().item()
        total_samples += y_batch.size(0)

    avg_loss = epoch_loss / total_samples
    accuracy = total_correct / total_samples

    epoch_losses.append(avg_loss)
    epoch_accuracies.append(accuracy)

    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# Plotting
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epoch_losses, label="Loss")
plt.title("Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epoch_accuracies, label="Accuracy", color='green')
plt.title("Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1, Loss: 0.7222, Accuracy: 0.3380
Epoch 2, Loss: 0.7208, Accuracy: 0.3485
Epoch 3, Loss: 0.7225, Accuracy: 0.3392


KeyboardInterrupt: 

# Model with PEC

In [55]:
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates."""
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis."""
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT."""
    for i in range(0, nqubits - 1, 2):  # Even indices
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Odd indices
        qml.CNOT(wires=[i, i + 1])

# Define noise model for PEC
class NoiseModel:
    """Simple noise model for PEC implementation."""
    def __init__(self, n_qubits, depolarizing_prob=0.01, readout_error_prob=0.02):
        self.n_qubits = n_qubits
        self.depolarizing_prob = depolarizing_prob
        self.readout_error_prob = readout_error_prob

    def characterize_noise(self):
        """
        Characterize the noise channels in the system.
        Returns a dictionary of noise parameters.
        """
        # In a real system, you would run calibration circuits to estimate these
        noise_params = {
            "depolarizing": self.depolarizing_prob,
            "readout_error": self.readout_error_prob,
            # Could include more noise types like amplitude damping, phase damping, etc.
        }
        return noise_params

    def compute_quasiprobabilities(self):
        """
        Compute quasiprobabilities for inverse noise channels.
        These would normally be derived from tomography experiments.

        Returns:
            - gamma: scaling factor
            - quasiprobs: dictionary of operations and their quasiprobabilities
        """
        # Simplified model: For each CNOT gate, we need to apply a correction
        # with probability p and do nothing with probability (1-p)
        p = self.depolarizing_prob

        # Compute the gamma factor (increases variance)
        gamma = 1 + 6 * p  # Example formula, real value depends on actual noise

        # Compute quasiprobabilities for inverse channels
        # This is a simplified model for depolarizing noise
        quasiprobs = {
            "I": 1 - 3*p,      # Identity with positive probability
            "X": p,            # Pauli X with positive probability
            "Y": p,            # Pauli Y with positive probability
            "Z": p             # Pauli Z with positive probability
        }

        return gamma, quasiprobs

# Applying corrections for PEC
def apply_pec_correction(wire, quasiprobs):
    """
    Apply a randomly selected correction operation based on quasiprobabilities.
    In practice, this would be done through random sampling of operations.

    Args:
        wire: The target qubit wire
        quasiprobs: Dictionary of operations and their quasiprobabilities
    """
    # Random selection would be done here in the actual implementation
    # For demonstration, we include all possible corrections with their weights

    # Apply Pauli X correction
    if quasiprobs["X"] > 0:
        qml.PauliX(wire)

    # Apply Pauli Y correction
    if quasiprobs["Y"] > 0:
        qml.PauliY(wire)

    # Apply Pauli Z correction
    if quasiprobs["Z"] > 0:
        qml.PauliZ(wire)

def pec_entangling_layer(nqubits, quasiprobs):
    """
    Entangling layer with PEC corrections applied.

    Args:
        nqubits: Number of qubits
        quasiprobs: Quasiprobabilities for correction operations
    """
    # Apply the original entangling layer
    for i in range(0, nqubits - 1, 2):  # Even indices
        qml.CNOT(wires=[i, i + 1])
        # Apply corrections after each CNOT
        apply_pec_correction(i, quasiprobs)
        apply_pec_correction(i+1, quasiprobs)

    for i in range(1, nqubits - 1, 2):  # Odd indices
        qml.CNOT(wires=[i, i + 1])
        # Apply corrections after each CNOT
        apply_pec_correction(i, quasiprobs)
        apply_pec_correction(i+1, quasiprobs)

# Define the original circuit without PEC
@qml.qnode(dev, interface="torch")
def quantum_net_no_pec(q_input_features, q_weights_flat):
    """
    Variational quantum circuit without PEC.
    """
    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from |+> state
    H_layer(n_qubits)

    # Embed input features
    RY_layer(q_input_features)

    # Apply trainable layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Return expectation values
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)]

# Define the circuit with PEC
@qml.qnode(dev, interface="torch")
def quantum_net_with_pec(q_input_features, q_weights_flat, gamma):
    """
    Variational quantum circuit with PEC.

    Args:
        q_input_features: Input features for embedding
        q_weights_flat: Flattened weights for variational layers
        gamma: Scaling factor for PEC

    Returns:
        List of expectation values
    """
    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Initialize noise model for PEC
    noise_model = NoiseModel(n_qubits)
    _, quasiprobs = noise_model.compute_quasiprobabilities()

    # Start from |+> state
    H_layer(n_qubits)

    # Embed input features
    RY_layer(q_input_features)

    # Apply trainable layers with PEC
    for k in range(q_depth):
        pec_entangling_layer(n_qubits, quasiprobs)
        RY_layer(q_weights[k])

    # Return expectation values (no scaling here - we'll do it outside the circuit)
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)]

def quantum_net(q_input_features, q_weights_flat):
    """
    Wrapper function implementing PEC for the quantum circuit.

    During training (grad enabled), applies PEC.
    During inference (grad disabled), uses the original circuit.

    Args:
        q_input_features: Input features
        q_weights_flat: Flattened weights

    Returns:
        List of expectation values
    """
    # Use PEC during training, original circuit during inference
    apply_pec = torch.is_grad_enabled()

    if apply_pec:
        # Initialize noise model for PEC
        noise_model = NoiseModel(n_qubits)
        gamma, _ = noise_model.compute_quasiprobabilities()

        # Call the PEC-enabled circuit
        pec_expectations = quantum_net_with_pec(q_input_features, q_weights_flat, gamma)

        # Convert each expectation to tensor/float and scale by gamma
        scaled_expectations = []
        for exp in pec_expectations:
            # Convert to float/tensor before scaling
            exp_value = exp.item() if hasattr(exp, 'item') else float(exp)
            scaled_value = exp_value * gamma
            scaled_expectations.append(scaled_value)

        return scaled_expectations
    else:
        # Use the original circuit without PEC
        return quantum_net_no_pec(q_input_features, q_weights_flat)

class PECQuantumNet(nn.Module):
    """
    Torch module for the dressed quantum network.
    """
    def __init__(self, input_shape=51, dataset_idx=1):
        super().__init__()
        self.n_qubits = n_qubits
        self.q_depth = q_depth
        self.q_delta = 0.01
        # self.pre_net = nn.Linear(input_shape, self.n_qubits)
        self.q_params = nn.Parameter(self.q_delta * torch.randn(self.q_depth * self.n_qubits))
        # self.post_net = nn.Linear(self.n_qubits, 7)

        if (dataset_idx == 1) or (dataset_idx == 2):
            self.pre_net = nn.Sequential(
                            nn.Linear(input_shape, 64),
                            nn.ReLU(),
                            nn.Linear(64, 64),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(64, 32),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(32, 16),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(16, 8)
                        )
        elif dataset_idx == 3:
            self.pre_net = nn.Sequential(
                            nn.Linear(input_shape, 16),
                            nn.ReLU(),
                            nn.Linear(16, 16),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(16, 8)
                        )

        self.post_net = nn.Sequential(
                            nn.Linear(n_qubits, 8),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(8, 4),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(4, 1)
                        )

    def forward(self, input_features):
        """
        Forward pass through the dressed quantum network.
        """
        # Preprocessing input to reduce dimensions
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply quantum circuit
        q_out = []
        for elem in q_in:
            elem = elem.clone().detach().to(self.q_params.device)  # Ensure it's on the correct device and detached
            q_out_elem = quantum_net(elem, self.q_params)  # Output as list

            # Convert list to tensor if it's not already
            q_out_elem_tensor = torch.tensor(q_out_elem, dtype=torch.float32, device=self.q_params.device)

            # Apply linear transformation to match 2 output classes (if needed)
            # q_out_elem = torch.nn.Linear(q_out_elem_tensor.size(-1), 2)(q_out_elem_tensor)  # Adjust size to 2
            q_out_elem = self.post_net(q_out_elem_tensor)

            q_out.append(q_out_elem)

        # Stack and process
        q_out = torch.stack(q_out, dim=0)  # Ensure this is stacked correctly
        # print("q_out after loop:", q_out.shape)

        # q_out = q_out.to(self.q_params.device, dtype=torch.float32, requires_grad=True)

        return torch.sigmoid(q_out)

In [56]:
model_IEEE_PEC = PECQuantumNet(input_shape=50, dataset_idx=1)

In [57]:
epoch_losses = []
epoch_accuracies = []

for epoch in range(20):
    model_IEEE_PEC.train()
    total_correct = 0
    total_samples = 0
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader_IEEE:
        X_batch = X_batch.to(model_IEEE_PEC.q_params.device)
        y_batch = y_batch.to(model_IEEE_PEC.q_params.device).unsqueeze(1)

        optimizer.zero_grad()
        output = model_IEEE_PEC(X_batch)
        loss = loss_fn(output, y_batch)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        epoch_loss += loss.item() * X_batch.size(0)

        # Convert predictions to binary (assuming binary classification)
        preds = (output > 0.5).float()
        total_correct += (preds == y_batch).sum().item()
        total_samples += y_batch.size(0)

    avg_loss = epoch_loss / total_samples
    accuracy = total_correct / total_samples

    epoch_losses.append(avg_loss)
    epoch_accuracies.append(accuracy)

    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# Plotting
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epoch_losses, label="Loss")
plt.title("Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epoch_accuracies, label="Accuracy", color='green')
plt.title("Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1, Loss: 0.5478, Accuracy: 0.9617
Epoch 2, Loss: 0.5481, Accuracy: 0.9617
Epoch 3, Loss: 0.5482, Accuracy: 0.9617


KeyboardInterrupt: 

# Model with VD

In [61]:
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates."""
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis."""
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT."""
    for i in range(0, nqubits - 1, 2):  # Even indices
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Odd indices
        qml.CNOT(wires=[i, i + 1])

# Define circuit for a single copy
def single_circuit(q_input_features, q_weights, n_qubits, q_depth):
    """
    Run the basic variational circuit once.

    Args:
        q_input_features: Input features
        q_weights: Weights for the circuit
        n_qubits: Number of qubits
        q_depth: Circuit depth
    """
    # Start from |+> state
    H_layer(n_qubits)

    # Embed input features
    RY_layer(q_input_features)

    # Apply trainable layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

@qml.qnode(dev, interface="torch")
def quantum_net_vd(q_input_features, q_weights_flat, copies=2):
    """
    Variational quantum circuit with Virtual Distillation error mitigation.

    Virtual distillation uses multiple copies of the quantum state to
    probabilistically project onto the highest eigenvalue component,
    which typically corresponds to the error-free state.

    Args:
        q_input_features: Input features
        q_weights_flat: Flat weights
        copies: Number of copies to use for distillation (default=2)

    Returns:
        Expectation values for each qubit
    """
    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # For a quantum computer we would prepare multiple copies in parallel
    # Since we're simulating, we'll do this sequentially and compute the overlap

    # First, prepare the basic circuit to get the noisy state
    single_circuit(q_input_features, q_weights, n_qubits, q_depth)

    # Get expectation values for each qubit
    expectations = [qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)]

    # Instead of attempting to convert directly to float, return the raw expectations
    # The transformation will be applied in the post-processing step
    return expectations

def apply_distillation(exp_values, copies=2):
    """
    Apply the virtual distillation transformation to expectation values.

    Args:
        exp_values: Tensor of expectation values
        copies: Number of copies to use for distillation

    Returns:
        Tensor of distilled expectation values
    """
    # Convert tensor to device-aware float values
    exp_values = exp_values.detach().clone()

    if copies == 2:
        # For 2 copies: (3x² - 1)/2 * sign(x)
        signs = torch.sign(exp_values)
        distilled = (3 * exp_values**2 - 1) / 2 * signs
    elif copies == 3:
        # For 3 copies: (5x³ - 3x)/2
        distilled = (5 * exp_values**3 - 3 * exp_values) / 2
    else:
        # For higher copies, default to 2 copies formula
        signs = torch.sign(exp_values)
        distilled = (3 * exp_values**2 - 1) / 2 * signs

    return distilled

def quantum_net(q_input_features, q_weights_flat):
    """
    Wrapper function implementing Virtual Distillation for the quantum circuit.

    During training (grad enabled), applies Virtual Distillation.
    During inference (grad disabled), uses the original circuit without distillation.

    Args:
        q_input_features: Input features
        q_weights_flat: Flattened weights

    Returns:
        List of expectation values
    """
    # Check if we're in training mode
    if torch.is_grad_enabled():
        # Use Virtual Distillation in training mode
        exp_values = quantum_net_vd(q_input_features, q_weights_flat, copies=2)
        # Convert the returned expectations to a tensor
        exp_tensor = torch.stack(exp_values) if not isinstance(exp_values, torch.Tensor) else exp_values
        # Apply distillation
        return apply_distillation(exp_tensor, copies=2)
    else:
        # Use the original circuit in eval mode
        # We reshape weights first
        q_weights = q_weights_flat.reshape(q_depth, n_qubits)

        # Define a basic circuit without distillation
        @qml.qnode(dev, interface="torch")
        def basic_circuit(q_input_features, q_weights_flat):
            q_weights = q_weights_flat.reshape(q_depth, n_qubits)

            # Start from |+> state
            H_layer(n_qubits)

            # Embed input features
            RY_layer(q_input_features)

            # Apply trainable layers
            for k in range(q_depth):
                entangling_layer(n_qubits)
                RY_layer(q_weights[k])

            # Return expectation values
            return [qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)]

        # Run without distillation
        return basic_circuit(q_input_features, q_weights_flat)

class VDQuantumNet(nn.Module):
    """
    Torch module for the dressed quantum network.
    """
    def __init__(self, input_shape=51, dataset_idx=1):
        super().__init__()
        self.n_qubits = n_qubits
        self.q_depth = q_depth
        self.q_delta = 0.01
        # self.pre_net = nn.Linear(input_shape, self.n_qubits)
        self.q_params = nn.Parameter(self.q_delta * torch.randn(self.q_depth * self.n_qubits))
        # self.post_net = nn.Linear(self.n_qubits, 7)

        if (dataset_idx == 1) or (dataset_idx == 2):
            self.pre_net = nn.Sequential(
                            nn.Linear(input_shape, 64),
                            nn.ReLU(),
                            nn.Linear(64, 64),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(64, 32),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(32, 16),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(16, 8)
                        )
        elif dataset_idx == 3:
            self.pre_net = nn.Sequential(
                            nn.Linear(input_shape, 16),
                            nn.ReLU(),
                            nn.Linear(16, 16),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(16, 8)
                        )

        self.post_net = nn.Sequential(
                            nn.Linear(n_qubits, 8),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(8, 4),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(4, 1)
                        )

    def forward(self, input_features):
        """
        Forward pass through the dressed quantum network.
        """
        # Preprocessing input to reduce dimensions
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply quantum circuit
        q_out = []
        for elem in q_in:
            elem = elem.clone().detach().to(self.q_params.device)  # Ensure it's on the correct device and detached
            q_out_elem = quantum_net(elem, self.q_params)  # Output as list or tensor

            # Ensure output is a tensor
            if not isinstance(q_out_elem, torch.Tensor):
                q_out_elem = torch.tensor(q_out_elem, dtype=torch.float32, device=self.q_params.device)

            # Apply post-processing
            q_out_elem = self.post_net(q_out_elem)
            q_out.append(q_out_elem)

        # Stack and process
        q_out = torch.stack(q_out)

        return torch.sigmoid(q_out)

In [62]:
model_IEEE_VD = VDQuantumNet(input_shape=50, dataset_idx=1)

In [63]:
epoch_losses = []
epoch_accuracies = []

for epoch in range(20):
    model_IEEE_VD.train()
    total_correct = 0
    total_samples = 0
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader_IEEE:
        X_batch = X_batch.to(model_IEEE_VD.q_params.device)
        y_batch = y_batch.to(model_IEEE_VD.q_params.device).unsqueeze(1)

        optimizer.zero_grad()
        output = model_IEEE_VD(X_batch)
        loss = loss_fn(output, y_batch)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        epoch_loss += loss.item() * X_batch.size(0)

        # Convert predictions to binary (assuming binary classification)
        preds = (output > 0.5).float()
        total_correct += (preds == y_batch).sum().item()
        total_samples += y_batch.size(0)

    avg_loss = epoch_loss / total_samples
    accuracy = total_correct / total_samples

    epoch_losses.append(avg_loss)
    epoch_accuracies.append(accuracy)

    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# Plotting
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epoch_losses, label="Loss")
plt.title("Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epoch_accuracies, label="Accuracy", color='green')
plt.title("Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1, Loss: 0.5929, Accuracy: 0.9233
Epoch 2, Loss: 0.5932, Accuracy: 0.9213
Epoch 3, Loss: 0.5919, Accuracy: 0.9248


KeyboardInterrupt: 